In [17]:
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime

In [11]:
base_folder = r"C:\Users\nikik\citibike_tripdata\citibike-dashboard\2022-citibike-tripdata"

In [12]:
filepaths = []
for root, dirs, files in os.walk(base_folder):
    for f in files:
        if f.lower().endswith(".csv"):
            filepaths.append(os.path.join(root, f))


In [13]:
dfs = []

for fp in filepaths:
    for chunk in pd.read_csv(fp, chunksize=100_000, low_memory=False):
        dfs.append(chunk)
    break  # only first file for demonstration


In [14]:
df = pd.concat(dfs, ignore_index=True)

In [15]:
df.shape

(1000000, 13)

# Task 5
# This code imports and joins the CitiBike 2022 trip data in an efficient way.
# Since the dataset is split across multiple CSV files stored in monthly subfolders,
# a recursive directory scan (os.walk) is used to automatically collect all CSV file paths.
# Each CSV file is read in smaller chunks using pandas.read_csv with the chunksize parameter.
# Chunked reading is necessary because the full dataset is very large and loading it at once
# could exceed system memory.
# The chunks (or selected files, if limited for demonstration) are then combined using
# pandas.concat to form a single DataFrame. This approach is scalable, memory-safe,
# and well suited for handling large datasets stored across multiple files.

In [19]:
TOKEN = "ZmYgqrmeDyjyAjJWzLjHdBbGeVxVpUzj"  
STATION_ID = "GHCND:USW00014732"       

url = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"
    "datasetid=GHCND&datatypeid=TAVG&limit=1000&"
    f"stationid={STATION_ID}&startdate=2022-01-01&enddate=2022-12-31"
)

In [20]:
r = requests.get(url, headers={"token": TOKEN})
print("Status:", r.status_code)

r.raise_for_status()  # will raise an error if request faile
d = r.json()          # same as json.loads(r.text)

Status: 200


In [21]:
results = d.get("results", [])
print("Records returned:", len(results))

# Keeping only what needed
df_weather = pd.DataFrame(results)[["date", "datatype", "value", "station"]]

# Converting date and temperature
df_weather["date"] = pd.to_datetime(df_weather["date"]).dt.date
df_weather["tavg_c"] = df_weather["value"] / 10.0  # tenths of °C → °C :contentReference[oaicite:4]{index=4}

# Optional: keeping only final columns
df_weather = df_weather[["date", "tavg_c", "station"]].sort_values("date")

df_weather.head(), df_weather.tail()

Records returned: 365


(         date  tavg_c            station
 0  2022-01-01    11.6  GHCND:USW00014732
 1  2022-01-02    11.4  GHCND:USW00014732
 2  2022-01-03     1.4  GHCND:USW00014732
 3  2022-01-04    -2.7  GHCND:USW00014732
 4  2022-01-05     3.2  GHCND:USW00014732,
            date  tavg_c            station
 360  2022-12-27    -0.7  GHCND:USW00014732
 361  2022-12-28     3.4  GHCND:USW00014732
 362  2022-12-29     6.4  GHCND:USW00014732
 363  2022-12-30     9.3  GHCND:USW00014732
 364  2022-12-31     8.2  GHCND:USW00014732)

In [22]:
out_path = "laguardia_weather_2022_tavg.csv"
df_weather.to_csv(out_path, index=False)
out_path

'laguardia_weather_2022_tavg.csv'

In [24]:
df_weather = pd.read_csv("laguardia_weather_2022_tavg.csv")

# Ensure date is a DATE (not datetime string)
df_weather["date"] = pd.to_datetime(df_weather["date"]).dt.date

In [25]:
df_weather.head()

,date,tavg_c,station
0,2022-01-01,11.6,GHCND:USW00014732
1,2022-01-02,11.4,GHCND:USW00014732
2,2022-01-03,1.4,GHCND:USW00014732
3,2022-01-04,-2.7,GHCND:USW00014732
4,2022-01-05,3.2,GHCND:USW00014732


In [26]:
pd.read_csv(filepaths[0], nrows=5).columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [27]:
out_csv = "citibike_2022_with_weather.csv"

# delete old output if it exists
if os.path.exists(out_csv):
    os.remove(out_csv)

first_write = True

for fp in sorted(filepaths):
    print("Processing:", os.path.basename(fp))
    
    for chunk in pd.read_csv(fp, chunksize=200_000, low_memory=False):
        
        # Create merge key from trip start time
        chunk["started_at"] = pd.to_datetime(chunk["started_at"], errors="coerce")
        chunk["date"] = chunk["started_at"].dt.date
        
        merged = chunk.merge(df_weather, how="left", on="date")
        
        merged.to_csv(out_csv, mode="a", index=False, header=first_write)
        first_write = False

Processing: 202201-citibike-tripdata_1.csv
Processing: 202201-citibike-tripdata_2.csv
Processing: 202202-citibike-tripdata_1.csv
Processing: 202202-citibike-tripdata_2.csv
Processing: 202203-citibike-tripdata_1.csv
Processing: 202203-citibike-tripdata_2.csv
Processing: 202204-citibike-tripdata_1.csv
Processing: 202204-citibike-tripdata_2.csv
Processing: 202204-citibike-tripdata_3.csv
Processing: 202205-citibike-tripdata_1.csv
Processing: 202205-citibike-tripdata_2.csv
Processing: 202205-citibike-tripdata_3.csv
Processing: 202206-citibike-tripdata_1.csv
Processing: 202206-citibike-tripdata_2.csv
Processing: 202206-citibike-tripdata_3.csv
Processing: 202206-citibike-tripdata_4.csv
Processing: 202207-citibike-tripdata_1.csv
Processing: 202207-citibike-tripdata_2.csv
Processing: 202207-citibike-tripdata_3.csv
Processing: 202207-citibike-tripdata_4.csv
Processing: 202208-citibike-tripdata_1.csv
Processing: 202208-citibike-tripdata_2.csv
Processing: 202208-citibike-tripdata_3.csv
Processing:

In [28]:
print("Saved merged file:", out_csv)

Saved merged file: citibike_2022_with_weather.csv
